In [3]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from nltk.tokenize import word_tokenize #import library nltk - tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import library sastrawi untuk
from textblob import TextBlob
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
import np

nltk.download('punkt')

def load_data():
#     data = pd.read_csv('korona_22Des.csv',header=None,names=['post_id','user_name','tweet'])
    data = pd.read_excel('TESTING.xlsx',header=None,names=['posting_id','user_name','tweet'])
    return data

# 
# type(load_data())
tweet_df = load_data()

# # print(tweet_df.head())

df=pd.DataFrame(tweet_df)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\binnafis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['posting_id', 'user_name', 'tweet'],
 [1340639259472999936,
  'ravenskye',
  "b'[DISLINE 2020]\\n\\nBuat temen aktivis kampus atau mahasiswa yang pengen tau asiknya ormawa di kampus, gabung live IG b\\xe2\\x80\\xa6 https://t.co/VD4wpXT8S4'"],
 [1340639070712560128,
  'REVOLUSIÃ°Å¸Å’Â¡Ã¯Â¸Â\uf18f',
  "b'25T buat demokrasi agak lumayanlah\\ntp korona ngrampok 92T dr KEN https://t.co/8ZCwsk3zIE'"],
 [1340638637826830080,
  'madamia',
  "b'Anti-vaksin berisik vaksin korona ini cuma ngada-ngada buat kontrol umat manusia karena kalau bisa bikin vaksin sec\\xe2\\x80\\xa6 https://t.co/EnMv5K1Okp'"],
 [1340638635683500032,
  'Ã°Å¸â€\xa0Â\uf181Ã°Å¸â€¦Â¸Ã°Å¸â€¦Â´Ã°Å¸â€¦Â·Ã°Å¸â€¦Â´Ã°Å¸Â¦â€¹',
  "b'@utiindah @BTS_twt Harapan nya semoga bangtan army selalu bersama, army juga makin kompak, bangtan sehat sellau jug\\xe2\\x80\\xa6 https://t.co/J0IpNvzE7n'"],
 [1340638420289270016,
  'Douglas Kiplagat',
  "b'Hio mnyama inashtua kama korona... Ati kaka whaaat\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82\\xf

# SLANK

In [30]:
full_text = [] #deklarasi variabel komentar pada list
full_text = df['tweet'].values.tolist() #masukan data kedalam list
full_text #cetak data dalam list

['tweet',
 "b'[DISLINE 2020]\\n\\nBuat temen aktivis kampus atau mahasiswa yang pengen tau asiknya ormawa di kampus, gabung live IG b\\xe2\\x80\\xa6 https://t.co/VD4wpXT8S4'",
 "b'25T buat demokrasi agak lumayanlah\\ntp korona ngrampok 92T dr KEN https://t.co/8ZCwsk3zIE'",
 "b'Anti-vaksin berisik vaksin korona ini cuma ngada-ngada buat kontrol umat manusia karena kalau bisa bikin vaksin sec\\xe2\\x80\\xa6 https://t.co/EnMv5K1Okp'",
 "b'@utiindah @BTS_twt Harapan nya semoga bangtan army selalu bersama, army juga makin kompak, bangtan sehat sellau jug\\xe2\\x80\\xa6 https://t.co/J0IpNvzE7n'",
 "b'Hio mnyama inashtua kama korona... Ati kaka whaaat\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82@RashidAbdalla #NipasheWikendi'",
 "b'Pas korona masih sedikit, nih RS tu 3 RS rujukan utama di jateng bareng sama RS Kariadi &amp; Purwokerto (lupa nama RSn\\xe2\\x80\\xa6 https://t.co/l1Bgt1iMdU'",
 "b'Moon tuh CA pertama gw setelah gw balik ke rp karena lagi korona, gw dulu pernah ge 

In [31]:
slankword={
    "skrg":"sekarang",
    "ni":"ini","tp":"tetapi","ms":"masih",
    "iso":"bisa",
    "syaanggg":"sayang",
    "sebelom":"sebelum",
    "tbtb":"lalu",
    "muali":"mulai",
    "tak":"tidak",
    "yt":"youtube","nimbrung":"mengikuti","ngasal":"asal",
    "sampe":"sampai","hp":"handphone","aja":"saja","nihhh":"nih","pastiii":"pasti","kagaa":"tidak","blm":"belum",
    "anjim":"anjing",
    "yg":"yang",
    "diyt":"di youtube",
    "gk":"tidak",
    "aing":"saya",
    "hny":"hanya",
    "dr":"dari",
    "gak":"tidak",
    "plg":"paling",
    "sdh":"sudah",
    "adlh":"adalah",
    "vs":"versus",
    "ndak":"tidak",
    "ntr":"nanti",
    "klo":"kalau",
    "dlu":"dulu",
    "krn":"karena","kadrun":"kadal gurun",
    "gw":"saya","sssaja":"saja",
    "gua":"saya",
    "gmn":"bagaimana",
    "gw":"saya",
    "tdk":"tidak","kaga":"tidak",
    "kpd":"kepada",
    "unk":"untuk",
    "t4gar":"tagar",
    "lg":"lagi",
    "rs":"rumah sakit",
    "trus":"terus","go out:(":"pergi sad",
    "w":"saya",
    "udah":"sudah","susaaah":"susah","bgt":"sangat",
    "dsr":"dasar",
    "srh":"suruh",
    "dg":"dengan","skuyy":"","lgsg":"",
    "jg":"juga","trs":"terus",
    "covid-19":"korona",
    "sars-cov-2":"korona",
    "tgr":"tagar",
    "aq":"saya","ssaja":"saja",
    "gr2":"karena","dgn":"dengan",
    "kovid-19":"korona",
    "gue":"saya","alloh":"Allah",
    "jd":"jadi","plis jng":"mohon jangan",
    "gpp":"tidak apa","onlen":"online",
    "ato":"atau",
    "ga":"tidak","hhhhhh":"sad",
    "engtidak":"tidak","xe x xa":"",
    "covid 19":"korona","kiyowoo":"imut",
    "kelar":"selesai",
    "kek":"seperti",
    "pankapanlah":"nanti","uu":"undang-undang",
    "pesepedamarak":"pesepeda marak",
    "cepatt":"cepat","drpd":"daripada",
    "ntar":"nanti","sampek":"sampai","udh":"sudah","grgr":"karena","ngtidak":"tidak","ak":"saya","tpi":"tetapi","knp":"kenapa"
}

## hasil dari slankword

In [32]:
for index, text in enumerate(full_text):
    text = text.lower()
    splitlines = word_tokenize(text)
    for word in splitlines:
        if word in slankword:
            text = text.replace(word, str(slankword[word]));
            full_text[index] = text
full_text

['tweet',
 "b'[DISLINE 2020]\\n\\nBuat temen aktivis kampus atau mahasiswa yang pengen tau asiknya ormawa di kampus, gabung live IG b\\xe2\\x80\\xa6 https://t.co/VD4wpXT8S4'",
 "b'25t buat demokrasi agak lumayanlah\\ntp korona ngrampok 92t dari ken https://t.co/8zcwsk3zie'",
 "b'Anti-vaksin berisik vaksin korona ini cuma ngada-ngada buat kontrol umat manusia karena kalau bisa bikin vaksin sec\\xe2\\x80\\xa6 https://t.co/EnMv5K1Okp'",
 "b'@utiindah @BTS_twt Harapan nya semoga bangtan army selalu bersama, army juga makin kompak, bangtan sehat sellau jug\\xe2\\x80\\xa6 https://t.co/J0IpNvzE7n'",
 "b'Hio mnyama inashtua kama korona... Ati kaka whaaat\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82@RashidAbdalla #NipasheWikendi'",
 "b'pas korona masih sedikit, nih rumah sakit tu 3 rumah sakit rujukan utama di jateng bareng sama rumah sakit kariadi &amp; purwokerto (lupa nama rumah sakitn\\xe2\\x80\\xa6 https://t.co/l1bgt1imdu'",
 "b'moon tuh ca pertama saya setelah saya balik ke

In [34]:
def slank(texts): #slank menggunakan fungsi
    text_list =[]
    for index, text in enumerate(texts):
        text = text.lower()
#         splitlines = word_tokenize(text)
        text = word_tokenize(text)
        for word in splitlines:
            if word in slankword:
                text = text.replace(word, str(slankword[word]))
                text[index] = text
    return texts
# print(slank(full_text))
df['slank_text']=slank(full_text)

In [35]:
df['slank_text'].values

array(['tweet',
       "b'[DISLINE 2020]\\n\\nBuat temen aktivis kampus atau mahasiswa yang pengen tau asiknya ormawa di kampus, gabung live IG b\\xe2\\x80\\xa6 https://t.co/VD4wpXT8S4'",
       "b'25t buat demokrasi agak lumayanlah\\ntp korona ngrampok 92t dari ken https://t.co/8zcwsk3zie'",
       "b'Anti-vaksin berisik vaksin korona ini cuma ngada-ngada buat kontrol umat manusia karena kalau bisa bikin vaksin sec\\xe2\\x80\\xa6 https://t.co/EnMv5K1Okp'",
       "b'@utiindah @BTS_twt Harapan nya semoga bangtan army selalu bersama, army juga makin kompak, bangtan sehat sellau jug\\xe2\\x80\\xa6 https://t.co/J0IpNvzE7n'",
       "b'Hio mnyama inashtua kama korona... Ati kaka whaaat\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82@RashidAbdalla #NipasheWikendi'",
       "b'pas korona masih sedikit, nih rumah sakit tu 3 rumah sakit rujukan utama di jateng bareng sama rumah sakit kariadi &amp; purwokerto (lupa nama rumah sakitn\\xe2\\x80\\xa6 https://t.co/l1bgt1imdu'",
       "

In [60]:
def remove_pattern(tweet,pattern):
    print(tweet)
    r=re.findall(pattern,tweet)
    for i in r:
        tweet=re.sub(i,'',tweet)
    return tweet
df['remove_user']=np.vectorize(remove_pattern)(df['slank_text'],"@[\w]*")
# df['slank_text']

b'a kokarin da gwamnatin jihar kaduna ke yi na takaita yaduwar korona bairos, datidak gobe litinin 21 tidak watan dizamba 2\xe2\x80\xa6 https://t.co/cnfadhphbq'
b'a kokarin da gwamnatin jihar kaduna ke yi na takaita yaduwar korona bairos, datidak gobe litinin 21 tidak watan dizamba 2\xe2\x80\xa6 https://t.co/cnfadhphbq'
b'@AmitShah Ab Kahan gaya korona'
b'@ftrsrrn abis korona   dah...'
b'ada komen yang bilang, izone bagus dan gede, cuma masalahnya mereka di jepang mulu\xf0\x9f\xa4\xa3\nwaktu mau rilis lagi di korea, ke\xe2\x80\xa6 https://t.co/pczaua7ugj'
b'@ajiandiri afc menyerahkan ke pssi, kompetisi baru 3 minggu jalan korona terus berhenti force majeur.. emg pssi nya saja tidak bener, kocak'
b'aku dari sedikit banyak fans echiendi yang bertahan sampai saat ini..\nsedih banget banyak yang ilang atau rest..\nfansite\xe2\x80\xa6 https://t.co/ui3ulegwl2'
b'Aku suka Padang, bahasa nya lucu dan aku paham dengan bahasanya. Insyaallah taun depan wisata ke padang, kalau duni\xe2\x80\xa6 htt

In [37]:
df[['remove_user']].values

array([['tweet'],
       ["b'[DISLINE 2020]\\n\\nBuat temen aktivis kampus atau mahasiswa yang pengen tau asiknya ormawa di kampus, gabung live IG b\\xe2\\x80\\xa6 https://t.co/VD4wpXT8S4'"],
       ["b'25t buat demokrasi agak lumayanlah\\ntp korona ngrampok 92t dari ken https://t.co/8zcwsk3zie'"],
       ["b'Anti-vaksin berisik vaksin korona ini cuma ngada-ngada buat kontrol umat manusia karena kalau bisa bikin vaksin sec\\xe2\\x80\\xa6 https://t.co/EnMv5K1Okp'"],
       ["b'  Harapan nya semoga bangtan army selalu bersama, army juga makin kompak, bangtan sehat sellau jug\\xe2\\x80\\xa6 https://t.co/J0IpNvzE7n'"],
       ["b'Hio mnyama inashtua kama korona... Ati kaka whaaat\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82 #NipasheWikendi'"],
       ["b'pas korona masih sedikit, nih rumah sakit tu 3 rumah sakit rujukan utama di jateng bareng sama rumah sakit kariadi &amp; purwokerto (lupa nama rumah sakitn\\xe2\\x80\\xa6 https://t.co/l1bgt1imdu'"],
       ["b'moon tuh ca pe

In [38]:
def remove(tweet):
    # Membuang angka
    tweet=re.sub('[0-9]+','',tweet)

    # Membuang stock market stiker seperti GE
    tweet=re.sub(r'\$\w*b','',tweet)

    # membuang style retweet RT text "RT"
    tweet = re.sub(r'^RT[\s]+','',tweet)
    
    # membuang hashtag cuman tanda # nya doang
    tweet= re.sub(r'#','',tweet)
    return tweet
df['remove_RT']=df['remove_user'].apply(lambda x:remove(x))

In [48]:
df['remove_RT']

id                                                 tweet
1      b'[DISLINE ]\n\nBuat temen aktivis kampus atau...
2      b't buat demokrasi agak lumayanlah\ntp korona ...
3      b'Anti-vaksin berisik vaksin korona ini cuma n...
4      b'  Harapan nya semoga bangtan army selalu ber...
                             ...                        
251                            b' Korona nempel di baju'
252    b'aku dari sedikit banyak fans echiendi yang b...
253              b'   dufan rame pas korona gini tidak?'
254    b'  korona maburr, bs liburan dengan tenang, k...
255                      b' kan korona gtu g msup sekol'
Name: remove_RT, Length: 256, dtype: object

In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\binnafis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
# import stopword
from nltk.corpus import stopwords
stopwords_indonesia=stopwords.words('indonesian')

# import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory=StemmerFactory()
stemmer =factory.create_stemmer()

# tokenize
from nltk.tokenize import TweetTokenizer

# Emoji Senang
emoticons_happy= set([
    ':-)',':)',';)',':o)',':]',':3',':c)',':>','=]','8)','=}',
    ':^)',':-D',':D','8-D','8D','x-D','XD','X-D','xD','wkwkwkw','haha'
])

# Emoji Sedih
emoticons_sad =set([
    ':l',':-/','>:/',':S','>:]',':@',':-(',':[',':-||','-l',':<',
    ':-[',':-<','=\\','=/','>:(',':(','>_<','hhh'
])

# all emoticons (hapy+sad)
emoticons=emoticons_happy.union(emoticons_sad)


def clean_tweets(tweet):
#     remove stock markert like $GE
    tweet=re.sub(r'\$\w*','',tweet)
    
#     remove old style retweet text 'RT'
    tweet=re.sub(r'^RT[\s]+','',tweet)

#     remove old style retweet text 'RT'
    tweet=re.sub(r'^rt[\s]+','',tweet)
    
#     remove hyperlinks
    tweet=re.sub(r'https?:\/\/.*[\r\n]*','',tweet)
    
#     remove hashtag
#     hanya membuang bagian hash # dari kalimat
    tweet=re.sub(r'#','',tweet)
    
#     remove coma
    tweet=re.sub(r',',' ',tweet)
    
#     remove angka
    tweet=re.sub('[0-9]+','',tweet)

#     tokenize tweets
    tokenizer=TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
    tweet_tokens=tokenizer.tokenize(tweet)
    
    tweets_clean=[]
    for word in tweet_tokens:
        if(word not in stopwords_indonesia and #remove stropwords
              word not in emoticons and #removes emoticon
                  word not in string.punctuation): #remove punctuation
             # tweets_clean.append(word)
            stem_word=stemmer.stem(word) #remove word
            tweets_clean.append(stem_word)
            
    return tweets_clean
    

In [74]:
df['tweet_clean']=df['remove_RT'].apply(lambda x: clean_tweets(x))

KeyError: 'remove_RT'

In [72]:
df['tweet_clean']
df

KeyError: 'tweet_clean'

In [49]:
def remove_punct(text):
    text=' '.join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] =df['tweet_clean'].apply(lambda x:remove_punct(x))

In [50]:
df.sort_values("Tweet",inplace=True)

In [51]:
komentar = [] #deklarasi variabel komentar pada list
komentar = df['Tweet'].values.tolist() #masukan data kedalam list

In [ ]:
# df.drop(df.columns[[0,1,2,3,4]],axis=1,inplace=True)

In [52]:
df.drop_duplicates(subset="Tweet",keep='first',inplace=True)

In [57]:
df['Tweet'].values

array(['b a kokarin da gwamnatin jihar kaduna yi na takaita yaduwar korona bairos datidak gobe litinin watan dizamba xe x xa',
       'b ab kahan gaya korona', 'b abis korona dah',
       'b ada komen bilang izone bagus gede jepang mulu xf xf xa xa nwaktu rilis korea xe x xa',
       'b afc serah pssi kompetisi minggu jalan korona henti force majeur emg pssi nya bener kocak',
       'b aku fans echiendi tahan nsedih banget ilang rest nfansite xe x xa',
       'b aku suka padang bahasa nya lucu paham bahasa insyaallah taun wisata padang duni xe x xa',
       'b anak rem y bro bener antara korona',
       'b anda korona ku anak zumba yoga',
       'b andai korona muka bumi indah nan permai',
       'b angkat korona ya kak',
       'b anjir kesel nih ya tmn kls smpe ranking an murid krg ranking xe x xa',
       'b anjlok tidakra korona mudah mudah anjlok ubah aamin xf xf xa xb xf xf xa xb xf xf x xf',
       'b anti-vaksin berisik vaksin korona ngada-ngada kontrol umat manusia bikin vaksi

In [58]:
df.to_csv('data_korona_prepocessing.csv',encoding='utf8',index=True)
df

post_id                    user_name  \
155  1340589275297890048      Abdallah Yunus Abdallah   
204  1340581198565809920                  Tarik tyagi   
196  1340582086281989888                     ayayawae   
166  1340586651274390016                   another 12   
9    1340637100039749888              Jan etes dewasa   
..                   ...                          ...   
173  1340585332434060032  Ã¢â„¢Â¡ÃŒÂ·ÃŒÂ·ÃŒÂ· baginda   
34   1340630129366629888                      Mr hans   
201  1340582077960499968               BBC News Hausa   
110  1340600173748199936             Just call me dee   
id            posting_id                    user_name   

                                                 tweet  \
155  b'A kokarin da Gwamnatin Jihar Kaduna ke yi na...   
204                  b'@AmitShah Ab Kahan gaya korona'   
196          b'@ftrsrrn Abis korona lgsg skuyy dah...'   
166  b'Ada komen yg bilang, izone bagus dan gede, c...   
9    b'@Ajiandiri Afc menyerahkan ke pssi, kompetis...   
..                                                 ...   
173  b'@134340RUN \xf0\x9f\x98\x94 ayo hilangkn kor...   
34   b'@Beritasatu Ya harus bisa di maklumi ini kan...   
201  b"@elrufai ya umarci ma'aikatan Kaduna su fara...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                            slank_text  \
155  b'a kokarin da gwamnatin jihar kaduna ke yi na...   
204                  b'@AmitShah Ab Kahan gaya korona'   
196                   b'@ftrsrrn abis korona   dah...'   
166  b'ada komen yang bilang, izone bagus dan gede,...   
9    b'@ajiandiri afc menyerahkan ke pssi, kompetis...   
..                                                 ...   
173  b'@134340RUN \xf0\x9f\x98\x94 ayo hilangkn kor...   
34   b'@Beritasatu Ya harus bisa di maklumi ini kan...   
201  b"@elrufai ya umarci ma'aikatan Kaduna su fara...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                           remove_user  \
155  b'a kokarin da gwamnatin jihar kaduna ke yi na...   
204                           b' Ab Kahan gaya korona'   
196                           b' abis korona   dah...'   
166  b'ada komen yang bilang, izone bagus dan gede,...   
9    b' afc menyerahkan ke pssi, kompetisi baru 3 m...   
..                                                 ...   
173           b' \xf0\x9f\x98\x94 ayo hilangkn korona'   
34   b' Ya harus bisa di maklumi ini kan masalah du...   
201  b" ya umarci ma'aikatan Kaduna su fara aiki da...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                             remove_RT  \
155  b'a kokarin da gwamnatin jihar kaduna ke yi na...   
204                           b' Ab Kahan gaya korona'   
196                           b' abis korona   dah...'   
166  b'ada komen yang bilang, izone bagus dan gede,...   
9    b' afc menyerahkan ke pssi, kompetisi baru  mi...   
..                                                 ...   
173                 b' \xf\xf\x\x ayo hilangkn korona'   
34   b' Ya harus bisa di maklumi ini kan masalah du...   
201  b" ya umarci ma'aikatan Kaduna su fara aiki da...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                           tweet_clean  \
155  [b a, kokarin, da, gwamnatin, jihar, kaduna, y...   
204                       [b, ab, kahan, gaya, korona]   
196                           [b, abis, korona, dah, ]   
166  [b ada, komen, bilang, izone, bagus, gede, jep...   
9    [b, afc, serah, pssi, kompetisi, minggu, jalan...   
..                                                 ...   
173           [b, xf, xf, x, x, ayo, hilangkn, korona]   
34   [b, ya, maklum, dunia, nasional, looo, wabah, ...   
201  [b, ya, umarci, m

# SENTIMEN ANALISIS


In [ ]:
def load_data():
    data = pd.read_csv('data_korona_prepocessing.csv')
    return data

tweet_df = load_data()

# print(tweet_df.head())

df=pd.DataFrame(tweet_df[['Tweet']])

full_text = [] #deklarasi variabel komentar pada list
full_text = df['Tweet'].values.tolist() #masukan data kedalam list
full_text #cetak data dalam list

In [61]:
analysis = TextBlob(text)
analysis=analysis.translate(to="en")

# Step 2 function untuk mengecek sentiment dari tweet.
def polarity(analysis):
    threshold = 0
    if analysis == threshold:
        return 'Neutral'
    elif analysis > threshold:
        return 'Positive'
    else:
        return 'Negative'


# The x in the lambda function is a row (because I set axis=1)
# Apply iterates the function accross the dataframe's rows
df['polarity'] = df.apply(lambda x: TextBlob(x['Tweet']).sentiment.polarity, axis=1)
df['subjectivity'] = df.apply(lambda x: TextBlob(x['Tweet']).sentiment.subjectivity, axis=1)
df['sentiment']=df.apply(lambda x: polarity(TextBlob(x['Tweet']).sentiment.polarity), axis=1)

In [62]:
df.to_csv('data_korona_sentimen.csv',encoding='utf8',index=True)
df

post_id                    user_name  \
155  1340589275297890048      Abdallah Yunus Abdallah   
204  1340581198565809920                  Tarik tyagi   
196  1340582086281989888                     ayayawae   
166  1340586651274390016                   another 12   
9    1340637100039749888              Jan etes dewasa   
..                   ...                          ...   
173  1340585332434060032  Ã¢â„¢Â¡ÃŒÂ·ÃŒÂ·ÃŒÂ· baginda   
34   1340630129366629888                      Mr hans   
201  1340582077960499968               BBC News Hausa   
110  1340600173748199936             Just call me dee   
id            posting_id                    user_name   

                                                 tweet  \
155  b'A kokarin da Gwamnatin Jihar Kaduna ke yi na...   
204                  b'@AmitShah Ab Kahan gaya korona'   
196          b'@ftrsrrn Abis korona lgsg skuyy dah...'   
166  b'Ada komen yg bilang, izone bagus dan gede, c...   
9    b'@Ajiandiri Afc menyerahkan ke pssi, kompetis...   
..                                                 ...   
173  b'@134340RUN \xf0\x9f\x98\x94 ayo hilangkn kor...   
34   b'@Beritasatu Ya harus bisa di maklumi ini kan...   
201  b"@elrufai ya umarci ma'aikatan Kaduna su fara...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                            slank_text  \
155  b'a kokarin da gwamnatin jihar kaduna ke yi na...   
204                  b'@AmitShah Ab Kahan gaya korona'   
196                   b'@ftrsrrn abis korona   dah...'   
166  b'ada komen yang bilang, izone bagus dan gede,...   
9    b'@ajiandiri afc menyerahkan ke pssi, kompetis...   
..                                                 ...   
173  b'@134340RUN \xf0\x9f\x98\x94 ayo hilangkn kor...   
34   b'@Beritasatu Ya harus bisa di maklumi ini kan...   
201  b"@elrufai ya umarci ma'aikatan Kaduna su fara...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                           remove_user  \
155  b'a kokarin da gwamnatin jihar kaduna ke yi na...   
204                           b' Ab Kahan gaya korona'   
196                           b' abis korona   dah...'   
166  b'ada komen yang bilang, izone bagus dan gede,...   
9    b' afc menyerahkan ke pssi, kompetisi baru 3 m...   
..                                                 ...   
173           b' \xf0\x9f\x98\x94 ayo hilangkn korona'   
34   b' Ya harus bisa di maklumi ini kan masalah du...   
201  b" ya umarci ma'aikatan Kaduna su fara aiki da...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                             remove_RT  \
155  b'a kokarin da gwamnatin jihar kaduna ke yi na...   
204                           b' Ab Kahan gaya korona'   
196                           b' abis korona   dah...'   
166  b'ada komen yang bilang, izone bagus dan gede,...   
9    b' afc menyerahkan ke pssi, kompetisi baru  mi...   
..                                                 ...   
173                 b' \xf\xf\x\x ayo hilangkn korona'   
34   b' Ya harus bisa di maklumi ini kan masalah du...   
201  b" ya umarci ma'aikatan Kaduna su fara aiki da...   
110  b'Yg ada diotak seminggu ini adlh\nPernikahan ...   
id                                               tweet   

                                           tweet_clean  \
155  [b a, kokarin, da, gwamnatin, jihar, kaduna, y...   
204                       [b, ab, kahan, gaya, korona]   
196                           [b, abis, korona, dah, ]   
166  [b ada, komen, bilang, izone, bagus, gede, jep...   
9    [b, afc, serah, pssi, kompetisi, minggu, jalan...   
..                                                 ...   
173           [b, xf, xf, x, x, ayo, hilangkn, korona]   
34   [b, ya, maklum, dunia, nasional, looo, wabah, ...   
201  [b, ya, umarci, m

In [ ]:
df['polarity'].values

# KNN 

In [70]:
data = pd.read_csv('data_korona_sentimen.csv',names=['Tweet','sentiment'])
X = data.Tweet
y = data.sentiment

#Using CountVectorizer to convert text into tokens/features
vect = CountVectorizer(stop_words='english', ngram_range = (1,1), max_df = 0.95, min_df = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

#Using training data to transform text into counts of features for each message
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_test_dtm = vect.transform(X_test)

In [68]:
#Accuracy using KNN Model
KNN = KNeighborsClassifier(n_neighbors =3)
KNN.fit(X_train_dtm, y_train)
y_pred = KNN.predict(X_test_dtm)
print('\nK Nearest Neighbors (NN = 3)')
print('Accuracy Score: ',metrics.accuracy_score(y_test,y_pred),sep='')
print('Confusion Matrix: ',metrics.confusion_matrix(y_test,y_pred), sep = '\n')


K Nearest Neighbors (NN = 3)
Accuracy Score: 0.9523809523809523
Confusion Matrix: 
[[ 0  1  0  0]
 [ 0 60  0  0]
 [ 0  1  0  0]
 [ 0  1  0  0]]


In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         1
     Neutral       0.92      1.00      0.96        58
    Positive       0.00      0.00      0.00         4

    accuracy                           0.92        63
   macro avg       0.31      0.33      0.32        63
weighted avg       0.85      0.92      0.88        63



g:\labs\sentimen-django\.venv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# HITUNG BOBOT TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
opinion=[]
opinion=df['Tweet'].values.tolist()
opinion


In [ ]:
#untuk menghitung jumlah kata yang telah di steming
cv = CountVectorizer()
term_fit = cv.fit(opinion)

print (len(term_fit.vocabulary_))


In [ ]:
print (term_fit.vocabulary_)

In [ ]:
print (term_fit.get_feature_names()) #mengurutkan berdasarkan urutkan nama 

In [ ]:
#kolom pertama ini berarti jumlah dokumen
#kolom kedua berarti letak katanya
#kolom ketiga hasil dari tf

term_frequency_all = term_fit.transform(komentar)
print (term_frequency_all)

In [ ]:
komen_tf = opinion[1] #memanggil kata pada index ke 1
print (komen_tf)

In [ ]:
term_frequency = term_fit.transform([komen_tf]) #hanya menampilkan hasil document 1
print (term_frequency)

In [ ]:
dokumen = term_fit.transform(opinion) #hasil perhitungan tf idf dalam 1 doc
tfidf_transformer = TfidfTransformer().fit(dokumen)
print (tfidf_transformer.idf_)

tfidf=tfidf_transformer.transform(term_frequency)
print (tfidf) #hasil manual dengan sistem pyhton

In [ ]:
hasil_tfidf=[]
hasil_tfidf=tfidf.values.tolist()
hasil_tfidf